In [44]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; 
las; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

In [32]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [43]:
llm = ChatOllama(model="exaone3.5:2.4b")

# 나라명 -> 제일 유명한 음식
country = input("나라를 입력해주세요 :")
country_prompt = PromptTemplate(
    template = "{country}에서 제일 유명한 음식은 무엇인가요? 음식 이름만 말해주세요",
    input_variables = ["country"])
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(country_prompt.invoke({"country":country})))

# country_chain 생성
country_chain = country_prompt | llm | output_parser

# 음식 -> 레시피
food = country_chain.invoke({"country":country})
food_prompt = PromptTemplate(
    template = "{food}의 레시피를 알려줘. 레시피만 간략하게 요약해서 말해줘",
    input_variables = ["food"])
output_parser1 = StrOutputParser()
output_parser1.invoke(llm.invoke(food_prompt.invoke({"food":food})))

# food_chain 생성
food_chain = food_prompt | llm | output_parser
food_chain.invoke({"food":food})

# 복합체인 : 나라명 -> 그 나라에서 제일 유명한 음식(country_chain)
                    # 음식 -> 레시피(food_chain)
final_chain = {"food":RunnablePassthrough()} | \
                {"country":country_chain} | food_chain
print(f"{country}에서 가장 유명한 음식은 {food}입니다")
print(final_chain.invoke({"country":food}))

나라를 입력해주세요 :한국
한국에서 가장 유명한 음식은 비빔밥입니다
비빔밥 레시피 간략 요약:
- 주요 재료: 밥, 다양한 야채(시금치, 당근, 고사리 등), 고기(소고기나 돼지고기), 고추장 또는 간장 기반의 양념, 참기름, 깨
- 준비: 밥 덜어 그릇에 담고, 각 야채를 볶거나 익힌 후 넣기
- 조리: 소고기 또는 돼지고기를 볶아 섞음, 양념(고추장+간장)에 다진 마늘과 후추로 맛내기
- 마무리: 위 재료들을 모두 섞어 한 그릇에 담고, 참기름과 깨 뿌려 완성
